In [13]:
import seaborn as sns
import matplotlib.pyplot as plt
import streamlit as st
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np
#from urllib.parse import quote
# pip install openpyxl

## Puxando dados de quais ações estão na B3

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

url = "https://www.infomoney.com.br/cotacoes/empresas-b3/"
page = requests.get(url, headers=headers)
page

<Response [200]>

In [3]:
soup = bs(page.content, "html.parser")

In [4]:
dados = soup.find_all('td', class_ ='strong')

In [5]:
#Lista de Ações
tick1 = []

for tiks in dados:
    tick1.append(tiks.get_text().strip())

tickers = [item for item in tick1 if not (str(item).endswith('11') or str(item).endswith('12') or str(item).endswith('F') or str(item).endswith('34') or str(item).endswith('39') or str(item).endswith('31') or str(item).endswith('13') or str(item).endswith('33') or str(item).endswith('35'))]

In [6]:
len(tickers)

296

In [7]:
t1 = pd.DataFrame(columns = tickers)
t1

,ZAMP3,AERI3,ICBR3,DOTZ3,GOLL3,ARML3,MLAS3,CBAV3,TTEN3,NINJ3,...,TAEE3,SBSP3,GEPA4,GEPA3,CESP6,CESP5,CESP3,CMIG4,CMIG3,AFLT3


## Fazer uma base com os multiplos de todas as ações

In [104]:
for index, acao in enumerate(tickers):
    if index == 0:
        header1 = { 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
        url = f"https://www.fundamentus.com.br/detalhes.php?papel={acao}"
        page1 = requests.get(url, headers=header1)
        soup1 = bs(page1.content, "html.parser")
        dados1 = soup1.find_all('span', class_='txt')
        if dados1 == []:
            pass
        else:
            funds = []
            for valor in dados1:
                funds.append(valor.get_text().strip())

            delet = [28, 29, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 98, 99, 100]

            # Remover os itens usando pop() em um loop reverso para evitar problemas de índices
            for index in sorted(delet, reverse=True):
                funds.pop(index)

            indices = funds[::2]
            valores = funds[1::2]
            
            # Após obter os dados em formato de lista 'valores', substituir as vírgulas por pontos
            valores = [valor.replace(',', '.') for valor in valores]

            fundamentos = pd.DataFrame(columns=indices)
            fundamentos.columns.values[47] = "Lucro Líquido(3m)"
            fundamentos.columns.values[46] = "Lucro Líquido(12m)"
            fundamentos.columns.values[45] = "EBIT(3m)"
            fundamentos.columns.values[44] = "EBIT(12m)"
            fundamentos.columns.values[43] = "Receita Líquida(3m)"
            fundamentos.columns.values[42] = "Receita Líquida(12m)"
            fundamentos.loc[len(fundamentos)] = valores
    else:
        header1 = { 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
        url = f"https://www.fundamentus.com.br/detalhes.php?papel={acao}"
        page1 = requests.get(url, headers=header1)
        soup1 = bs(page1.content, "html.parser")
        dados1 = soup1.find_all('span', class_='txt')
        if dados1 == []:
            pass
        else:
            funds = []
            for valor in dados1:
                funds.append(valor.get_text().strip())
    
            delet = [28, 29, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 98, 99, 100]
    
            # Remover os itens usando pop() em um loop reverso para evitar problemas de índices
            for index in sorted(delet, reverse=True):
                funds.pop(index)
    
            indices = funds[::2]
            valores = funds[1::2]
    
            # Após obter os dados em formato de lista 'valores', substituir as vírgulas por pontos
            valores = [valor.replace(',', '.') for valor in valores]

            # Preencher a lista 'valores' com valores None até que ela tenha o mesmo tamanho das colunas de 'fundamentos'
            while len(valores) < len(fundamentos.columns):
                valores.append(None)

            # Criar um dicionário para os dados da nova linha
            novo_registro = dict(zip(fundamentos.columns, valores))

            # Adicionar o novo registro ao DataFrame
            fundamentos = fundamentos.append(novo_registro, ignore_index=True)

In [105]:
fundamentos

,Papel,Cotação,Tipo,Data últ cot,Empresa,Min 52 sem,Setor,Max 52 sem,Subsetor,Vol $ méd (2m),...,Disponibilidades,Dív. Líquida,Ativo Circulante,Patrim. Líq,Receita Líquida(12m),Receita Líquida(3m),EBIT(12m),EBIT(3m),Lucro Líquido(12m),Lucro Líquido(3m)
0,ZAMP3,4.31,ON,28/07/2023,ZAMP S.A. ON,3.72,Hoteis e Restaurantes,8.80,Restaurante e Similares,8.252.880,...,457.109.000,641.507.000,905.494.000,1.427.870.000,3.728.690.000,885.209.000,94.674.800,-2.756.010,-79.583.000,-55.168.000
1,AERI3,1.19,ON NM,28/07/2023,AERIS ON NM,0.88,Máquinas e Equipamentos,3.47,Máq. e Equip. Industriais,8.094.230,...,828.913.000,984.805.000,2.089.070.000,862.600.000,2.763.740.000,831.622.000,256.307.000,82.326.000,-116.161.000,-22.206.000
2,DOTZ3,1.34,ON NM,28/07/2023,DOTZ SA ON NM,0.76,Diversos,3.70,Programas de Fidelização,725.539,...,157.254.000,-47.720.000,201.544.000,-184.960.000,135.687.000,30.299.000,-59.739.000,-23.028.000,-94.034.000,-15.529.000
3,ARML3,12.86,ON NM,28/07/2023,ARMAC ON NM,9.35,Diversos,16.72,Aluguel de carros,7.784.120,...,869.390.000,1.335.850.000,1.255.490.000,1.139.730.000,1.047.830.000,291.491.000,351.268.000,93.645.000,146.849.000,26.222.000
4,MLAS3,3.13,ON NM,28/07/2023,MULTILASER ON NM,1.26,Computadores e Equipamentos,5.86,Computadores e Equipamentos,7.606.720,...,892.408.000,400.710.000,5.206.350.000,3.710.960.000,4.192.990.000,784.648.000,-398.795.000,-335.834.000,-423.679.000,-342.613.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,CESP5,25.00,PNA,25/03/2022,CESP CIA ENERGETICA SAO PAULO PNA,22.50,Energia Elétrica,26.99,Energia Elétrica,0,...,601.783.000,1.430.460.000,1.157.960.000,7.583.450.000,2.316.980.000,581.048.000,1.188.300.000,170.363.000,324.915.000,-114.000
272,CESP3,25.55,ON,25/03/2022,CESP CIA ENERGETICA SAO PAULO ON,22.23,Energia Elétrica,25.55,Energia Elétrica,0,...,601.783.000,1.430.460.000,1.157.960.000,7.583.450.000,2.316.980.000,581.048.000,1.188.300.000,170.363.000,324.915.000,-114.000
273,CMIG4,12.60,PN N1,28/07/2023,CEMIG PN N1,9.55,Energia Elétrica,12.86,Energia Elétrica,118.729.000,...,2.957.110.000,7.322.210.000,13.138.100.000,22.791.100.000,35.262.300.000,8.646.940.000,6.133.500.000,1.918.230.000,4.034.660.000,1.397.540.000
274,CMIG3,18.40,ON N1,28/07/2023,CEMIG ON N1,14.34,Energia Elétrica,20.37,Energia Elétrica,5.917.220,...,2.957.110.000,7.322.210.000,13.138.100.000,22.791.100.000,35.262.300.000,8.646.940.000,6.133.500.000,1.918.230.000,4.034.660.000,1.397.540.000
